# Air Quality Prediction - Preprocessing Pipeline
## Jakarta ISPU Data (2022-2025)

This notebook creates a comprehensive preprocessing pipeline for air quality prediction, including:
1. **Temporal Filtering & Population Extrapolation** - Filter data to 2022-2025 and extrapolate population
2. **Spatial Mapping** - Join river quality data using Haversine distance
3. **Advanced Feature Engineering** - Wind direction encoding, lag features, rolling windows
4. **Data Cleaning & Encoding** - Handle missing values, outliers, and encode target variable
5. **Final Output** - Merge all features and apply StandardScaler

In [1]:
# =============================================================================
# CELL 1: Import Libraries and Configuration
# =============================================================================
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression
import warnings
import os
import glob

warnings.filterwarnings('ignore')

# Configuration
DATA_PATH = "../penyisihan-datavidia-10/"
START_YEAR = 2022
END_YEAR = 2025

# Station coordinates (latitude, longitude) for Jakarta ISPU stations
# These are approximate coordinates for each monitoring station
STATION_COORDS = {
    'DKI1': {'name': 'Bundaran HI', 'lat': -6.1944, 'lon': 106.8229, 'area': 'Jakarta Pusat'},
    'DKI2': {'name': 'Kelapa Gading', 'lat': -6.1616, 'lon': 106.9070, 'area': 'Jakarta Utara'},
    'DKI3': {'name': 'Jagakarsa', 'lat': -6.3339, 'lon': 106.8235, 'area': 'Jakarta Selatan'},
    'DKI4': {'name': 'Lubang Buaya', 'lat': -6.2914, 'lon': 106.9017, 'area': 'Jakarta Timur'},
    'DKI5': {'name': 'Kebon Jeruk', 'lat': -6.1886, 'lon': 106.7633, 'area': 'Jakarta Barat'}
}

print("✓ Libraries imported successfully")
print(f"✓ Data path: {DATA_PATH}")
print(f"✓ Target years: {START_YEAR}-{END_YEAR}")

✓ Libraries imported successfully
✓ Data path: ../penyisihan-datavidia-10/
✓ Target years: 2022-2025


## Section 1: Load All Datasets

In [3]:
# =============================================================================
# CELL 2: Load and Combine ISPU Data (2022-2025)
# =============================================================================

def load_ispu_data(data_path):
    """
    Load and combine all ISPU (Air Quality Index) data files from 2022-2025.
    Standardizes column names across different file formats.
    
    Returns:
        pd.DataFrame: Combined ISPU data with standardized columns
    """
    ispu_files = glob.glob(os.path.join(data_path, "ISPU", "*.csv"))
    
    dfs = []
    for file in ispu_files:
        df = pd.read_csv(file)
        
        # Standardize column names across different file formats
        col_mapping = {
            'pm_10': 'pm_sepuluh',
            'so2': 'sulfur_dioksida',
            'co': 'karbon_monoksida',
            'o3': 'ozon',
            'no2': 'nitrogen_dioksida',
            'critical': 'parameter_pencemar_kritis',
            'categori': 'kategori',
            'lokasi_spku': 'stasiun'
        }
        df.rename(columns=col_mapping, inplace=True)
        
        # Handle the 'tanggal' column which may be in different formats
        # For 2024 and 2025 files, 'tanggal' is just the day number and 'bulan' is the month
        if 'bulan' in df.columns and 'tanggal' in df.columns:
            # Construct date from periode_data (YYYYMM), bulan, and tanggal (day)
            df['year'] = df['periode_data'].astype(str).str[:4].astype(int)
            df['month'] = df['bulan'].astype(int)
            df['day'] = df['tanggal'].astype(int)
            df['tanggal'] = pd.to_datetime(df[['year', 'month', 'day']])
        elif 'tanggal' in df.columns:
            # Handle Excel serial date format (numeric) and string dates
            # Try to convert - if it's a number > 40000, it's likely an Excel serial date
            def parse_date(val):
                if pd.isna(val):
                    return pd.NaT
                if isinstance(val, (int, float)) and val > 40000:
                    # Excel serial date - days since 1899-12-30
                    try:
                        return pd.Timestamp('1899-12-30') + pd.Timedelta(days=int(val))
                    except:
                        return pd.NaT
                try:
                    return pd.to_datetime(val)
                except:
                    return pd.NaT
            
            df['tanggal'] = df['tanggal'].apply(parse_date)
        
        # Extract station ID from station name if needed
        if 'stasiun' in df.columns:
            df['stasiun_id'] = df['stasiun'].astype(str).str.extract(r'(DKI\d)')[0]
        
        dfs.append(df)
    
    # Combine all dataframes
    df_ispu = pd.concat(dfs, ignore_index=True)
    
    # Extract station_id for consistent naming
    if 'stasiun_id' not in df_ispu.columns:
        df_ispu['stasiun_id'] = df_ispu['stasiun'].astype(str).str.extract(r'(DKI\d)')[0]
    
    # Ensure tanggal is datetime
    df_ispu['tanggal'] = pd.to_datetime(df_ispu['tanggal'], errors='coerce')
    
    # Drop rows with invalid dates
    df_ispu = df_ispu.dropna(subset=['tanggal'])
    
    # Extract year for filtering
    df_ispu['year'] = df_ispu['tanggal'].dt.year
    
    # Select and order relevant columns
    cols_to_keep = ['tanggal', 'stasiun_id', 'stasiun', 'pm_sepuluh', 'pm_duakomalima',
                    'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida',
                    'max', 'parameter_pencemar_kritis', 'kategori', 'year']
    
    existing_cols = [col for col in cols_to_keep if col in df_ispu.columns]
    df_ispu = df_ispu[existing_cols]
    
    # Drop duplicates based on date and station
    df_ispu = df_ispu.drop_duplicates(subset=['tanggal', 'stasiun_id'])
    
    return df_ispu.sort_values(['stasiun_id', 'tanggal']).reset_index(drop=True)

# Load ISPU data
df_ispu = load_ispu_data(DATA_PATH)
print(f"✓ ISPU data loaded: {df_ispu.shape[0]:,} records")
print(f"  Date range: {df_ispu['tanggal'].min().date()} to {df_ispu['tanggal'].max().date()}")
print(f"  Stations: {sorted([s for s in df_ispu['stasiun_id'].unique() if pd.notna(s)])}")
print(f"  Years available: {sorted(df_ispu['year'].unique())}")
df_ispu.head()

✓ ISPU data loaded: 16,686 records
  Date range: 2010-01-01 to 2025-08-31
  Stations: ['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']
  Years available: [np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025)]


,tanggal,stasiun_id,stasiun,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,max,parameter_pencemar_kritis,kategori,year
0,2010-01-01,DKI1,DKI1 (Bunderan HI),NaN,NaN,4,73,27,14,73,CO,SEDANG,2010
1,2010-01-02,DKI1,DKI1 (Bunderan HI),NaN,NaN,2,16,33,9,33,O3,BAIK,2010
2,2010-01-03,DKI1,DKI1 (Bunderan HI),NaN,NaN,2,19,20,9,27,PM10,BAIK,2010
3,2010-01-04,DKI1,DKI1 (Bunderan HI),NaN,NaN,2,16,15,6,22,PM10,BAIK,2010
4,2010-01-05,DKI1,DKI1 (Bunderan HI),NaN,NaN,2,17,15,8,25,PM10,BAIK,2010


In [4]:
# =============================================================================
# CELL 3: Load Weather Data (All Stations)
# =============================================================================

def load_weather_data(data_path):
    """
    Load and combine weather data from all 5 Jakarta monitoring stations.
    Maps file names to station IDs for joining with ISPU data.
    
    Returns:
        pd.DataFrame: Combined weather data with station IDs
    """
    weather_files = glob.glob(os.path.join(data_path, "cuaca-harian", "*.csv"))
    
    # Mapping of file patterns to station IDs
    station_file_mapping = {
        'dki1': 'DKI1',
        'dki2': 'DKI2', 
        'dki3': 'DKI3',
        'dki4': 'DKI4',
        'dki5': 'DKI5'
    }
    
    dfs = []
    for file in weather_files:
        df = pd.read_csv(file)
        
        # Determine station ID from filename
        filename_lower = os.path.basename(file).lower()
        station_id = None
        for pattern, sid in station_file_mapping.items():
            if pattern in filename_lower:
                station_id = sid
                break
        
        if station_id:
            df['stasiun_id'] = station_id
            
            # Rename time column to tanggal for consistency
            if 'time' in df.columns:
                df.rename(columns={'time': 'tanggal'}, inplace=True)
            
            df['tanggal'] = pd.to_datetime(df['tanggal'], errors='coerce')
            dfs.append(df)
    
    df_weather = pd.concat(dfs, ignore_index=True)
    
    # Simplify column names (remove units)
    col_rename = {
        'temperature_2m_max (°C)': 'temp_max',
        'temperature_2m_min (°C)': 'temp_min',
        'temperature_2m_mean (°C)': 'temp_mean',
        'precipitation_sum (mm)': 'precipitation_sum',
        'precipitation_hours (h)': 'precipitation_hours',
        'wind_speed_10m_max (km/h)': 'wind_speed_max',
        'wind_speed_10m_mean (km/h)': 'wind_speed_mean',
        'wind_speed_10m_min (km/h)': 'wind_speed_min',
        'wind_direction_10m_dominant (°)': 'wind_direction_10m_dominant',
        'winddirection_10m_dominant (°)': 'wind_direction_alt',
        'shortwave_radiation_sum (MJ/m²)': 'radiation_sum',
        'relative_humidity_2m_mean (%)': 'humidity_mean',
        'relative_humidity_2m_max (%)': 'humidity_max',
        'relative_humidity_2m_min (%)': 'humidity_min',
        'cloud_cover_mean (%)': 'cloud_cover_mean',
        'cloud_cover_max (%)': 'cloud_cover_max',
        'cloud_cover_min (%)': 'cloud_cover_min',
        'surface_pressure_mean (hPa)': 'pressure_mean',
        'surface_pressure_max (hPa)': 'pressure_max',
        'surface_pressure_min (hPa)': 'pressure_min',
        'wind_gusts_10m_max (km/h)': 'wind_gusts_max',
        'wind_gusts_10m_mean (km/h)': 'wind_gusts_mean',
        'wind_gusts_10m_min (km/h)': 'wind_gusts_min'
    }
    df_weather.rename(columns=col_rename, inplace=True)
    
    # Use primary wind direction column or fallback
    if 'wind_direction_10m_dominant' not in df_weather.columns and 'wind_direction_alt' in df_weather.columns:
        df_weather['wind_direction_10m_dominant'] = df_weather['wind_direction_alt']
    
    df_weather['year'] = df_weather['tanggal'].dt.year
    
    return df_weather.sort_values(['stasiun_id', 'tanggal']).reset_index(drop=True)

# Load weather data
df_weather = load_weather_data(DATA_PATH)
print(f"✓ Weather data loaded: {df_weather.shape[0]:,} records")
print(f"  Date range: {df_weather['tanggal'].min().date()} to {df_weather['tanggal'].max().date()}")
print(f"  Stations: {sorted(df_weather['stasiun_id'].unique())}")
print(f"  Columns: {list(df_weather.columns)}")

✓ Weather data loaded: 28,610 records
  Date range: 2010-01-01 to 2025-08-31
  Stations: ['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']
  Columns: ['tanggal', 'temp_max', 'temp_min', 'precipitation_sum', 'precipitation_hours', 'wind_speed_max', 'wind_direction_10m_dominant', 'radiation_sum', 'temp_mean', 'humidity_mean', 'cloud_cover_mean', 'pressure_mean', 'wind_gusts_max', 'wind_direction_alt', 'humidity_max', 'humidity_min', 'cloud_cover_max', 'cloud_cover_min', 'wind_gusts_mean', 'wind_speed_mean', 'wind_gusts_min', 'wind_speed_min', 'pressure_max', 'pressure_min', 'stasiun_id', 'year']


In [5]:
# =============================================================================
# CELL 4: Load Population, NDVI, River Quality, and Holiday Data
# =============================================================================

def load_population_data(data_path):
    """
    Load population data by kelurahan/kecamatan for 2013-2016.
    Aggregates by kecamatan and year for later extrapolation.
    """
    pop_file = os.path.join(data_path, "jumlah-penduduk", 
                            "data-jumlah-penduduk-provinsi-dki-jakarta-berdasarkan-kelompok-usia-dan-jenis-kelamin-tahun-2013-2021-komponen-data.csv")
    df_pop = pd.read_csv(pop_file)
    
    # Aggregate total population by kecamatan and year
    df_pop_agg = df_pop.groupby(['tahun', 'nama_kabupaten_kota', 'nama_kecamatan']).agg({
        'jumlah_penduduk': 'sum'
    }).reset_index()
    
    return df_pop_agg

def load_ndvi_data(data_path):
    """
    Load NDVI (Normalized Difference Vegetation Index) data.
    Higher NDVI indicates more vegetation, which can improve air quality.
    """
    ndvi_file = os.path.join(data_path, "NDVI (vegetation index)", "indeks-ndvi-jakarta.csv")
    df_ndvi = pd.read_csv(ndvi_file)
    df_ndvi['tanggal'] = pd.to_datetime(df_ndvi['tanggal'], errors='coerce')
    df_ndvi['year'] = df_ndvi['tanggal'].dt.year
    
    return df_ndvi

def load_river_data(data_path):
    """
    Load river water quality data with coordinates.
    Contains chemical parameters like pH, BOD, COD, DO, etc.
    """
    river_file = os.path.join(data_path, "kualitas-air-sungai", "data-kualitas-air-sungai-komponen-data.csv")
    df_river = pd.read_csv(river_file)
    
    # Pivot to get one row per sampling point with all parameters as columns
    # First, get unique sampling points
    df_river_pivot = df_river.pivot_table(
        index=['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel', 
               'nama_sungai', 'alamat', 'latitude', 'longitude'],
        columns='parameter',
        values='hasil_pengukuran',
        aggfunc='first'
    ).reset_index()
    
    # Flatten column names
    df_river_pivot.columns = [str(col) if isinstance(col, str) else col for col in df_river_pivot.columns]
    
    return df_river_pivot

def load_holiday_data(data_path):
    """
    Load national holiday and weekend data.
    Contains flags for weekends and national holidays.
    """
    holiday_file = os.path.join(data_path, "libur-nasional", "dataset-libur-nasional-dan-weekend.csv")
    df_holidays = pd.read_csv(holiday_file)
    df_holidays['tanggal'] = pd.to_datetime(df_holidays['tanggal'], errors='coerce')
    df_holidays['year'] = df_holidays['tanggal'].dt.year
    
    return df_holidays

# Load all supporting datasets
df_population = load_population_data(DATA_PATH)
df_ndvi = load_ndvi_data(DATA_PATH)
df_river = load_river_data(DATA_PATH)
df_holidays = load_holiday_data(DATA_PATH)

print(f"✓ Population data loaded: {df_population.shape[0]:,} records")
print(f"  Years: {sorted(df_population['tahun'].unique())}")
print(f"  Kecamatan count: {df_population['nama_kecamatan'].nunique()}")

print(f"\n✓ NDVI data loaded: {df_ndvi.shape[0]:,} records")
print(f"  Date range: {df_ndvi['tanggal'].min().date()} to {df_ndvi['tanggal'].max().date()}")

print(f"\n✓ River quality data loaded: {df_river.shape[0]:,} records")
print(f"  Unique sampling points: {df_river['titik_sampel'].nunique()}")
print(f"  Parameters: {df_river.columns.tolist()[8:]}")

print(f"\n✓ Holiday data loaded: {df_holidays.shape[0]:,} records")
print(f"  Date range: {df_holidays['tanggal'].min().date()} to {df_holidays['tanggal'].max().date()}")

✓ Population data loaded: 176 records
  Years: [np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016)]
  Kecamatan count: 44

✓ NDVI data loaded: 1,810 records
  Date range: 2009-12-19 to 2025-08-29

✓ River quality data loaded: 480 records
  Unique sampling points: 120
  Parameters: ['Amoniak', 'BOD', 'COD', 'Cd', 'Cr6', 'Cu', 'DO', 'F', 'Fecal Coliform', 'Fenol', 'H2S', 'Hg', 'Klorida', 'Klorin Bebas', 'Klorin Bebas n In Situ', 'MBAS', 'Minyak dan Lemak', 'Ni', 'Nitrat', 'Nitrit', 'Pb', 'Sianida', 'Sulfat', 'TDS', 'TSS', 'Total Coliform', 'Total N', 'Total P', 'Warna', 'Zn', 'pH']

✓ Holiday data loaded: 5,844 records
  Date range: 2010-01-01 to 2025-12-31


## Section 2: Temporal Filtering & Population Extrapolation

In [6]:
# =============================================================================
# CELL 5: Temporal Filtering Functions
# =============================================================================

def filter_by_year_range(df, start_year, end_year, date_col='tanggal', year_col='year'):
    """
    Filter dataframe to only include data within specified year range.
    
    Parameters:
        df: Input DataFrame
        start_year: Start year (inclusive)
        end_year: End year (inclusive)
        date_col: Name of date column
        year_col: Name of year column (will be created if not exists)
    
    Returns:
        Filtered DataFrame
    """
    df = df.copy()
    
    # Ensure year column exists
    if year_col not in df.columns and date_col in df.columns:
        df[year_col] = pd.to_datetime(df[date_col]).dt.year
    
    # Filter by year range
    mask = (df[year_col] >= start_year) & (df[year_col] <= end_year)
    filtered_df = df[mask].copy()
    
    return filtered_df

# Apply temporal filtering to datasets
df_ispu_filtered = filter_by_year_range(df_ispu, START_YEAR, END_YEAR)
df_weather_filtered = filter_by_year_range(df_weather, START_YEAR, END_YEAR)
df_holidays_filtered = filter_by_year_range(df_holidays, START_YEAR, END_YEAR, year_col='year')

print(f"✓ ISPU data filtered: {df_ispu_filtered.shape[0]:,} records ({START_YEAR}-{END_YEAR})")
print(f"  Date range: {df_ispu_filtered['tanggal'].min().date()} to {df_ispu_filtered['tanggal'].max().date()}")

print(f"\n✓ Weather data filtered: {df_weather_filtered.shape[0]:,} records ({START_YEAR}-{END_YEAR})")
print(f"  Date range: {df_weather_filtered['tanggal'].min().date()} to {df_weather_filtered['tanggal'].max().date()}")

print(f"\n✓ Holiday data filtered: {df_holidays_filtered.shape[0]:,} records ({START_YEAR}-{END_YEAR})")
print(f"  Date range: {df_holidays_filtered['tanggal'].min().date()} to {df_holidays_filtered['tanggal'].max().date()}")

✓ ISPU data filtered: 5,176 records (2022-2025)
  Date range: 2022-01-01 to 2025-08-31

✓ Weather data filtered: 6,695 records (2022-2025)
  Date range: 2022-01-01 to 2025-08-31

✓ Holiday data filtered: 1,461 records (2022-2025)
  Date range: 2022-01-01 to 2025-12-31


In [9]:
# =============================================================================
# CELL 6: Population Extrapolation using Linear Regression
# =============================================================================
"""
EXTRAPOLATION LOGIC:
--------------------
Population data is only available for 2013-2016. To estimate population for 2022-2025,
we use LINEAR EXTRAPOLATION at the CITY (kabupaten/kota) level:

1. Aggregate population by city and year first
2. For each city, fit a linear regression model: Population = a * Year + b
3. Use this model to predict population for target years (2022-2025)
4. This approach is more stable than kecamatan-level extrapolation

Note: Linear extrapolation over a large time gap (6+ years) may introduce errors.
For production models, consider using official population projections from BPS.
The erratic historical data (non-monotonic) suggests data quality issues which
we handle by using city-level aggregation for more stable trends.
"""

def extrapolate_population_by_city(df_pop, target_years):
    """
    Extrapolate population for target years using linear regression at city level.
    
    Parameters:
        df_pop: Population DataFrame with columns [tahun, nama_kabupaten_kota, jumlah_penduduk]
        target_years: List of years to predict population for
    
    Returns:
        DataFrame with extrapolated population for each city
    """
    # First aggregate to city level
    df_city = df_pop.groupby(['tahun', 'nama_kabupaten_kota']).agg({
        'jumlah_penduduk': 'sum'
    }).reset_index()
    
    extrapolated_data = []
    
    for kota in df_city['nama_kabupaten_kota'].unique():
        # Get historical data for this city
        city_data = df_city[df_city['nama_kabupaten_kota'] == kota].copy()
        city_data = city_data.sort_values('tahun')
        
        if len(city_data) < 2:
            continue
        
        # Prepare data for linear regression
        X = city_data['tahun'].values.reshape(-1, 1)
        y = city_data['jumlah_penduduk'].values
        
        # Fit linear regression
        model = LinearRegression()
        model.fit(X, y)
        
        # Get slope and intercept for understanding trend
        slope = model.coef_[0]
        intercept = model.intercept_
        
        # Predict for target years
        for year in target_years:
            predicted_pop = model.predict([[year]])[0]
            # Ensure population is positive - use last known value if prediction is negative
            if predicted_pop <= 0:
                predicted_pop = city_data['jumlah_penduduk'].iloc[-1]
            
            extrapolated_data.append({
                'tahun': year,
                'nama_kabupaten_kota': kota,
                'jumlah_penduduk': int(predicted_pop),
                'annual_growth_rate': slope / city_data['jumlah_penduduk'].mean() * 100,
                'is_extrapolated': True
            })
    
    return pd.DataFrame(extrapolated_data)

# Perform population extrapolation at city level
target_years = list(range(START_YEAR, END_YEAR + 1))
df_pop_extrapolated = extrapolate_population_by_city(df_population, target_years)

print(f"✓ Population extrapolated for years: {target_years}")
print(f"  Total records: {df_pop_extrapolated.shape[0]:,}")
print(f"  Cities: {df_pop_extrapolated['nama_kabupaten_kota'].unique().tolist()}")

# Show extrapolation summary
print("\n  Extrapolated population by city:")
pivot_table = df_pop_extrapolated.pivot(index='nama_kabupaten_kota', columns='tahun', values='jumlah_penduduk')
print(pivot_table)

✓ Population extrapolated for years: [2022, 2023, 2024, 2025]
  Total records: 24
  Cities: ['JAKARTA BARAT', 'JAKARTA PUSAT', 'JAKARTA SELATAN', 'JAKARTA TIMUR', 'JAKARTA UTARA', 'KAB.ADM.KEP.SERIBU']

  Extrapolated population by city:
tahun                   2022     2023     2024     2025
nama_kabupaten_kota                                    
JAKARTA BARAT        2332364  2332364  2332364  2332364
JAKARTA PUSAT        1311376  1328565  1345755  1362945
JAKARTA SELATAN      2699172  2767771  2836369  2904968
JAKARTA TIMUR        2442570  2382158  2321747  2261335
JAKARTA UTARA        1688957  1688957  1688957  1688957
KAB.ADM.KEP.SERIBU     35876    36909    37941    38974


In [10]:
# =============================================================================
# CELL 7: Create Kecamatan-to-Station Mapping
# =============================================================================
"""
MAPPING LOGIC:
--------------
Each ISPU monitoring station is located in a specific area of Jakarta.
We map kabupaten/kota (city) to the nearest station based on administrative area:
- DKI1 (Bundaran HI) -> Jakarta Pusat
- DKI2 (Kelapa Gading) -> Jakarta Utara  
- DKI3 (Jagakarsa) -> Jakarta Selatan
- DKI4 (Lubang Buaya) -> Jakarta Timur
- DKI5 (Kebon Jeruk) -> Jakarta Barat
"""

# Map kabupaten/kota to stations
KOTA_TO_STATION = {
    'JAKARTA PUSAT': 'DKI1',
    'JAKARTA UTARA': 'DKI2',
    'JAKARTA SELATAN': 'DKI3',
    'JAKARTA TIMUR': 'DKI4',
    'JAKARTA BARAT': 'DKI5',
    'KAB.ADM.KEP.SERIBU': 'DKI2'  # Map to nearest (north)
}

def map_population_to_stations(df_pop, kota_station_map):
    """
    Map population by station based on administrative area mapping.
    
    Parameters:
        df_pop: Population DataFrame with nama_kabupaten_kota column
        kota_station_map: Dictionary mapping kota names to station IDs
    
    Returns:
        DataFrame with population mapped to station by year
    """
    df = df_pop.copy()
    
    # Map kota to station
    df['stasiun_id'] = df['nama_kabupaten_kota'].map(kota_station_map)
    
    # Aggregate population by station and year (in case multiple kota map to same station)
    df_agg = df.groupby(['tahun', 'stasiun_id']).agg({
        'jumlah_penduduk': 'sum'
    }).reset_index()
    
    return df_agg

# Map extrapolated population to stations
df_pop_by_station = map_population_to_stations(df_pop_extrapolated, KOTA_TO_STATION)

print("✓ Population mapped to stations")
print("\nExtrapolated population by station (2022-2025):")
pivot = df_pop_by_station.pivot(index='stasiun_id', columns='tahun', values='jumlah_penduduk')
print(pivot)

✓ Population mapped to stations

Extrapolated population by station (2022-2025):
tahun          2022     2023     2024     2025
stasiun_id                                    
DKI1        1311376  1328565  1345755  1362945
DKI2        1724833  1725866  1726898  1727931
DKI3        2699172  2767771  2836369  2904968
DKI4        2442570  2382158  2321747  2261335
DKI5        2332364  2332364  2332364  2332364


## Section 3: Spatial Mapping using Haversine Distance

In [11]:
# =============================================================================
# CELL 8: Haversine Distance Function for Spatial Mapping
# =============================================================================
"""
HAVERSINE FORMULA:
------------------
The Haversine formula calculates the great-circle distance between two points
on a sphere given their latitude and longitude coordinates.

Formula:
    a = sin²(Δlat/2) + cos(lat1) × cos(lat2) × sin²(Δlon/2)
    c = 2 × atan2(√a, √(1-a))
    d = R × c

Where:
    - R = Earth's radius (6371 km)
    - Δlat = lat2 - lat1 (in radians)
    - Δlon = lon2 - lon1 (in radians)

This is used to find the nearest ISPU station for each river sampling point.
"""

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on Earth.
    
    Parameters:
        lat1, lon1: Latitude and longitude of first point (in degrees)
        lat2, lon2: Latitude and longitude of second point (in degrees)
    
    Returns:
        Distance in kilometers
    """
    # Earth's radius in kilometers
    R = 6371.0
    
    # Convert degrees to radians
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    lon1_rad = np.radians(lon1)
    lon2_rad = np.radians(lon2)
    
    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

def find_nearest_station(lat, lon, station_coords):
    """
    Find the nearest ISPU station to a given coordinate.
    
    Parameters:
        lat, lon: Coordinates of the point
        station_coords: Dictionary of station coordinates {station_id: {'lat': x, 'lon': y}}
    
    Returns:
        Tuple of (station_id, distance_km)
    """
    min_distance = float('inf')
    nearest_station = None
    
    for station_id, coords in station_coords.items():
        distance = haversine_distance(lat, lon, coords['lat'], coords['lon'])
        if distance < min_distance:
            min_distance = distance
            nearest_station = station_id
    
    return nearest_station, min_distance

# Test the function
test_lat, test_lon = -6.286182, 106.870626  # Kalibaru Timur river sample point
nearest, dist = find_nearest_station(test_lat, test_lon, STATION_COORDS)
print(f"✓ Haversine distance function created")
print(f"  Test: Point ({test_lat}, {test_lon}) -> Nearest station: {nearest} ({dist:.2f} km)")

✓ Haversine distance function created
  Test: Point (-6.286182, 106.870626) -> Nearest station: DKI4 (3.48 km)


In [12]:
# =============================================================================
# CELL 9: Map River Quality Data to ISPU Stations
# =============================================================================

def map_river_to_stations(df_river, station_coords):
    """
    Map each river sampling point to the nearest ISPU station using Haversine distance.
    
    Parameters:
        df_river: River quality DataFrame with latitude/longitude columns
        station_coords: Dictionary of station coordinates
    
    Returns:
        DataFrame with river data mapped to nearest stations
    """
    df = df_river.copy()
    
    # Find nearest station for each sampling point
    station_assignments = []
    distances = []
    
    for _, row in df.iterrows():
        lat, lon = row['latitude'], row['longitude']
        if pd.notna(lat) and pd.notna(lon):
            station, dist = find_nearest_station(lat, lon, station_coords)
            station_assignments.append(station)
            distances.append(dist)
        else:
            station_assignments.append(None)
            distances.append(None)
    
    df['stasiun_id'] = station_assignments
    df['distance_to_station_km'] = distances
    
    return df

# Map river data to stations
df_river_mapped = map_river_to_stations(df_river, STATION_COORDS)

print("✓ River quality data mapped to ISPU stations")
print(f"\n  Mapping summary:")
print(df_river_mapped.groupby('stasiun_id').agg({
    'titik_sampel': 'nunique',
    'distance_to_station_km': ['mean', 'min', 'max']
}).round(2))

# Aggregate river quality by station and period
# Select key water quality parameters
key_params = ['pH', 'BOD', 'COD', 'DO', 'TSS']
available_params = [p for p in key_params if p in df_river_mapped.columns]

df_river_agg = df_river_mapped.groupby(['periode_data', 'stasiun_id']).agg({
    **{param: 'mean' for param in available_params}
}).reset_index()

print(f"\n✓ River quality aggregated by station and period")
print(f"  Available parameters: {available_params}")
df_river_agg.head()

✓ River quality data mapped to ISPU stations

  Mapping summary:
           titik_sampel distance_to_station_km             
                nunique                   mean   min    max
stasiun_id                                                 
DKI1                 33                   4.31  1.13   8.04
DKI2                 21                   5.23  2.40   9.04
DKI3                 16                   4.13  0.52   7.91
DKI4                 21                   5.13  1.10   7.31
DKI5                 29                   5.75  2.11  11.36

✓ River quality aggregated by station and period
  Available parameters: ['pH', 'BOD', 'COD', 'DO', 'TSS']


,periode_data,stasiun_id,pH,BOD,COD,DO,TSS
0,2024,DKI1,7.181212,19.480227,104.645152,2.034058,64.423030
1,2024,DKI2,7.276426,29.529643,156.428571,1.811929,67.125000
2,2024,DKI3,7.079150,12.824219,57.618594,3.410164,40.265625
3,2024,DKI4,7.165417,17.854048,80.211190,2.802082,70.410714
4,2024,DKI5,7.179496,16.237241,79.688879,2.075152,34.750000


## Section 4: Advanced Feature Engineering

In [13]:
# =============================================================================
# CELL 10: Wind Direction Circular Encoding
# =============================================================================
"""
CIRCULAR ENCODING FOR WIND DIRECTION:
-------------------------------------
Wind direction is a circular/cyclical variable where 0° and 360° are the same.
Standard linear encoding doesn't capture this circularity.

Solution: Convert to sine and cosine components:
    Wind_sin = sin(direction × π/180)
    Wind_cos = cos(direction × π/180)

This transformation:
1. Preserves the circular nature (0° = 360°)
2. Creates continuous features that ML models can interpret
3. Captures both the north-south (cos) and east-west (sin) components

Example:
- North (0°): sin=0, cos=1
- East (90°): sin=1, cos=0
- South (180°): sin=0, cos=-1
- West (270°): sin=-1, cos=0
"""

def encode_wind_direction(df, wind_col='wind_direction_10m_dominant'):
    """
    Convert wind direction (degrees) to sine and cosine components.
    
    Parameters:
        df: DataFrame with wind direction column
        wind_col: Name of the wind direction column (in degrees)
    
    Returns:
        DataFrame with wind_sin and wind_cos columns added
    """
    df = df.copy()
    
    if wind_col in df.columns:
        # Convert degrees to radians
        direction_rad = df[wind_col] * np.pi / 180
        
        # Calculate sine and cosine components
        df['wind_sin'] = np.sin(direction_rad)
        df['wind_cos'] = np.cos(direction_rad)
    
    return df

# Apply to weather data
df_weather_encoded = encode_wind_direction(df_weather_filtered)

print("✓ Wind direction encoded to circular components")
print(f"  Sample encoding (first 5 rows):")
print(df_weather_encoded[['tanggal', 'stasiun_id', 'wind_direction_10m_dominant', 'wind_sin', 'wind_cos']].head())

✓ Wind direction encoded to circular components
  Sample encoding (first 5 rows):
        tanggal stasiun_id  wind_direction_10m_dominant  wind_sin  wind_cos
4383 2022-01-01       DKI1                          259 -0.981627 -0.190809
4384 2022-01-02       DKI1                          254 -0.961262 -0.275637
4385 2022-01-03       DKI1                          231 -0.777146 -0.629320
4386 2022-01-04       DKI1                          356 -0.069756  0.997564
4387 2022-01-05       DKI1                          168  0.207912 -0.978148


In [14]:
# =============================================================================
# CELL 11: Lag Features for Air Quality Data
# =============================================================================
"""
LAG FEATURES:
-------------
Lag features capture temporal dependencies in air quality data.
Air quality on a given day is often influenced by conditions on previous days.

We create:
- 1-day lag: Captures immediate short-term persistence
- 7-day lag: Captures weekly patterns (e.g., weekday vs weekend effects)

These are crucial for time-series forecasting as they provide the model
with historical context about pollution levels.
"""

def create_lag_features(df, columns, lags, group_col='stasiun_id', date_col='tanggal'):
    """
    Create lag features for specified columns.
    
    Parameters:
        df: DataFrame with time series data
        columns: List of columns to create lags for
        lags: List of lag periods (in days)
        group_col: Column to group by (for per-station lags)
        date_col: Date column for sorting
    
    Returns:
        DataFrame with lag features added
    """
    df = df.copy()
    df = df.sort_values([group_col, date_col])
    
    for col in columns:
        if col in df.columns:
            for lag in lags:
                lag_col_name = f"{col}_lag_{lag}d"
                df[lag_col_name] = df.groupby(group_col)[col].shift(lag)
    
    return df

# Define columns for lag features
lag_columns = ['pm_sepuluh', 'pm_duakomalima']
lag_periods = [1, 7]  # 1-day and 7-day lags

# Apply lag features to ISPU data
df_ispu_with_lags = create_lag_features(df_ispu_filtered, lag_columns, lag_periods)

print("✓ Lag features created")
print(f"  Columns: {lag_columns}")
print(f"  Lag periods: {lag_periods} days")
print(f"\n  Sample lag features (DKI1, first 10 rows):")
lag_display_cols = ['tanggal', 'stasiun_id', 'pm_sepuluh', 'pm_sepuluh_lag_1d', 'pm_sepuluh_lag_7d', 
                    'pm_duakomalima', 'pm_duakomalima_lag_1d', 'pm_duakomalima_lag_7d']
existing_cols = [c for c in lag_display_cols if c in df_ispu_with_lags.columns]
print(df_ispu_with_lags[df_ispu_with_lags['stasiun_id'] == 'DKI1'][existing_cols].head(10))

✓ Lag features created
  Columns: ['pm_sepuluh', 'pm_duakomalima']
  Lag periods: [1, 7] days

  Sample lag features (DKI1, first 10 rows):
        tanggal stasiun_id pm_sepuluh pm_sepuluh_lag_1d pm_sepuluh_lag_7d  \
1918 2022-09-13       DKI1         59               NaN               NaN   
1919 2022-10-07       DKI1         51                59               NaN   
1920 2022-10-18       DKI1         59                51               NaN   
1921 2022-12-01       DKI1         54                59               NaN   
1922 2022-12-02       DKI1         53                54               NaN   
1923 2022-12-03       DKI1         60                53               NaN   
1924 2022-12-04       DKI1         58                60               NaN   
1925 2022-12-05       DKI1         54                58                59   
1926 2022-12-06       DKI1         58                54                51   
1927 2022-12-07       DKI1         48                58                59   

     pm_duak

In [15]:
# =============================================================================
# CELL 12: Rolling Window Features for Weather Data
# =============================================================================
"""
ROLLING WINDOW FEATURES:
------------------------
Rolling window statistics smooth out daily variations and capture trends.
A 3-day moving average helps:
1. Reduce noise from day-to-day weather fluctuations
2. Capture the cumulative effect of weather conditions
3. Provide more stable features for model training

We apply this to precipitation_sum and temperature_2m_mean.
"""

def create_rolling_features(df, columns, window_size, group_col='stasiun_id', date_col='tanggal'):
    """
    Create rolling window features for specified columns.
    
    Parameters:
        df: DataFrame with time series data
        columns: List of columns to create rolling features for
        window_size: Window size for rolling statistics
        group_col: Column to group by
        date_col: Date column for sorting
    
    Returns:
        DataFrame with rolling features added
    """
    df = df.copy()
    df = df.sort_values([group_col, date_col])
    
    for col in columns:
        if col in df.columns:
            # Create rolling mean
            rolling_col_name = f"{col}_rolling_{window_size}d_mean"
            df[rolling_col_name] = df.groupby(group_col)[col].transform(
                lambda x: x.rolling(window=window_size, min_periods=1).mean()
            )
    
    return df

# Define columns for rolling features
rolling_columns = ['precipitation_sum', 'temp_mean']
window_size = 3  # 3-day moving average

# Apply rolling features to weather data
df_weather_with_rolling = create_rolling_features(df_weather_encoded, rolling_columns, window_size)

print(f"✓ Rolling window features created ({window_size}-day mean)")
print(f"  Columns: {rolling_columns}")
print(f"\n  Sample rolling features (DKI1, first 10 rows):")
rolling_display_cols = ['tanggal', 'stasiun_id', 'precipitation_sum', 'precipitation_sum_rolling_3d_mean',
                        'temp_mean', 'temp_mean_rolling_3d_mean']
print(df_weather_with_rolling[df_weather_with_rolling['stasiun_id'] == 'DKI1'][rolling_display_cols].head(10))

✓ Rolling window features created (3-day mean)
  Columns: ['precipitation_sum', 'temp_mean']

  Sample rolling features (DKI1, first 10 rows):
        tanggal stasiun_id  precipitation_sum  \
4383 2022-01-01       DKI1                0.3   
4384 2022-01-02       DKI1                1.2   
4385 2022-01-03       DKI1                0.2   
4386 2022-01-04       DKI1                0.1   
4387 2022-01-05       DKI1               15.9   
4388 2022-01-06       DKI1               11.4   
4389 2022-01-07       DKI1                7.9   
4390 2022-01-08       DKI1                3.4   
4391 2022-01-09       DKI1                4.6   
4392 2022-01-10       DKI1               16.1   

      precipitation_sum_rolling_3d_mean  temp_mean  temp_mean_rolling_3d_mean  
4383                           0.300000       27.1                  27.100000  
4384                           0.750000       27.3                  27.200000  
4385                           0.566667       27.4                  27.266667

In [16]:
# =============================================================================
# CELL 13: Time Features Extraction
# =============================================================================
"""
TIME FEATURES:
--------------
Extract temporal features that capture seasonal and weekly patterns:
1. month: Captures seasonal patterns (rainy/dry season in Jakarta)
2. is_weekend: Weekend traffic patterns differ from weekdays
3. is_holiday_nasional: National holidays affect traffic and industrial activity
"""

def extract_time_features(df, df_holidays, date_col='tanggal'):
    """
    Extract time-based features from date column.
    
    Parameters:
        df: DataFrame with date column
        df_holidays: Holiday DataFrame with is_weekend and is_holiday_nasional
        date_col: Name of date column
    
    Returns:
        DataFrame with time features added
    """
    df = df.copy()
    
    # Extract month
    df['month'] = df[date_col].dt.month
    
    # Merge with holiday data
    holiday_cols = [date_col, 'is_weekend', 'is_holiday_nasional']
    df_holidays_subset = df_holidays[[c for c in holiday_cols if c in df_holidays.columns]].copy()
    
    # Merge holidays
    df = df.merge(df_holidays_subset, on=date_col, how='left')
    
    # Fill missing values (in case some dates don't have holiday data)
    if 'is_weekend' in df.columns:
        df['is_weekend'] = df['is_weekend'].fillna(0).astype(int)
    else:
        df['is_weekend'] = df[date_col].dt.dayofweek.isin([5, 6]).astype(int)
    
    if 'is_holiday_nasional' in df.columns:
        df['is_holiday_nasional'] = df['is_holiday_nasional'].fillna(0).astype(int)
    
    return df

# Apply time features to weather data (main dataframe)
df_weather_with_time = extract_time_features(df_weather_with_rolling, df_holidays_filtered)

print("✓ Time features extracted")
print(f"  Features: month, is_weekend, is_holiday_nasional")
print(f"\n  Sample time features (first 10 rows):")
time_display_cols = ['tanggal', 'stasiun_id', 'month', 'is_weekend', 'is_holiday_nasional']
print(df_weather_with_time[time_display_cols].head(10))

✓ Time features extracted
  Features: month, is_weekend, is_holiday_nasional

  Sample time features (first 10 rows):
     tanggal stasiun_id  month  is_weekend  is_holiday_nasional
0 2022-01-01       DKI1      1           1                    1
1 2022-01-02       DKI1      1           1                    0
2 2022-01-03       DKI1      1           0                    0
3 2022-01-04       DKI1      1           0                    0
4 2022-01-05       DKI1      1           0                    0
5 2022-01-06       DKI1      1           0                    0
6 2022-01-07       DKI1      1           0                    0
7 2022-01-08       DKI1      1           1                    0
8 2022-01-09       DKI1      1           1                    0
9 2022-01-10       DKI1      1           0                    0


## Section 5: Data Cleaning & Encoding

In [17]:
# =============================================================================
# CELL 14: Handle Missing Values with Time-Series Interpolation
# =============================================================================
"""
MISSING VALUE HANDLING:
-----------------------
For pollutant columns, we use linear interpolation which:
1. Estimates missing values based on surrounding time points
2. Preserves temporal continuity in the data
3. Is more appropriate than mean imputation for time-series data

Linear interpolation: value = prev_value + (next_value - prev_value) * ratio
"""

def interpolate_pollutants(df, pollutant_cols, group_col='stasiun_id', date_col='tanggal'):
    """
    Handle missing values in pollutant columns using time-series linear interpolation.
    
    Parameters:
        df: DataFrame with pollutant columns
        pollutant_cols: List of pollutant column names
        group_col: Column to group by for interpolation
        date_col: Date column for sorting
    
    Returns:
        DataFrame with interpolated values
    """
    df = df.copy()
    df = df.sort_values([group_col, date_col])
    
    # First convert pollutant columns to numeric, handling string missing values like '-', '---'
    for col in pollutant_cols:
        if col in df.columns:
            # Convert to string first, then handle non-numeric values
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Interpolate within each station group
    for col in pollutant_cols:
        if col in df.columns:
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.interpolate(method='linear', limit_direction='both')
            )
    
    return df

# Define pollutant columns
pollutant_columns = ['pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 
                     'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max']

# Check missing values before interpolation
print("Missing values BEFORE interpolation:")
for col in pollutant_columns:
    if col in df_ispu_with_lags.columns:
        missing = df_ispu_with_lags[col].isna().sum()
        total = len(df_ispu_with_lags)
        print(f"  {col}: {missing:,} ({missing/total*100:.1f}%)")

# Apply interpolation
df_ispu_interpolated = interpolate_pollutants(df_ispu_with_lags, pollutant_columns)

print("\nMissing values AFTER interpolation:")
for col in pollutant_columns:
    if col in df_ispu_interpolated.columns:
        missing = df_ispu_interpolated[col].isna().sum()
        total = len(df_ispu_interpolated)
        print(f"  {col}: {missing:,} ({missing/total*100:.1f}%)")

Missing values BEFORE interpolation:
  pm_sepuluh: 186 (3.6%)
  pm_duakomalima: 53 (1.0%)
  sulfur_dioksida: 53 (1.0%)
  karbon_monoksida: 44 (0.9%)
  ozon: 43 (0.8%)
  nitrogen_dioksida: 66 (1.3%)
  max: 7 (0.1%)

Missing values AFTER interpolation:
  pm_sepuluh: 1 (0.0%)
  pm_duakomalima: 1 (0.0%)
  sulfur_dioksida: 1 (0.0%)
  karbon_monoksida: 1 (0.0%)
  ozon: 1 (0.0%)
  nitrogen_dioksida: 1 (0.0%)
  max: 1 (0.0%)


In [18]:
# =============================================================================
# CELL 15: Ordinal Encoding for Target Variable (kategori)
# =============================================================================
"""
ORDINAL ENCODING:
-----------------
The target variable 'kategori' represents air quality levels with a natural order:
- BAIK (Good): 0
- SEDANG (Moderate): 1
- TIDAK SEHAT (Unhealthy): 2
- SANGAT TIDAK SEHAT (Very Unhealthy): 3
- BERBAHAYA (Hazardous): 4

We use OrdinalEncoder to preserve this natural ordering, which is important
for ordinal regression or when treating this as a regression problem.
"""

def encode_target_variable(df, target_col='kategori'):
    """
    Encode target variable using ordinal encoding.
    
    Parameters:
        df: DataFrame with target column
        target_col: Name of target column
    
    Returns:
        Tuple of (DataFrame with encoded target, encoder object)
    """
    df = df.copy()
    
    # Define category order (from best to worst air quality)
    category_order = ['BAIK', 'SEDANG', 'TIDAK SEHAT', 'SANGAT TIDAK SEHAT', 'BERBAHAYA']
    
    # Standardize category names (uppercase, strip whitespace)
    if target_col in df.columns:
        df[target_col] = df[target_col].str.upper().str.strip()
    
    # Create ordinal encoder
    encoder = OrdinalEncoder(categories=[category_order], handle_unknown='use_encoded_value', unknown_value=-1)
    
    # Fit and transform
    if target_col in df.columns:
        # Handle NaN values
        mask = df[target_col].notna()
        df.loc[mask, 'kategori_encoded'] = encoder.fit_transform(
            df.loc[mask, [target_col]]
        ).flatten()
        
        # Fill NaN in encoded column
        df['kategori_encoded'] = df['kategori_encoded'].fillna(-1)
    
    return df, encoder

# Apply encoding
df_ispu_encoded, target_encoder = encode_target_variable(df_ispu_interpolated)

print("✓ Target variable encoded")
print("\n  Encoding mapping:")
for i, cat in enumerate(['BAIK', 'SEDANG', 'TIDAK SEHAT', 'SANGAT TIDAK SEHAT', 'BERBAHAYA']):
    print(f"    {cat}: {i}")

print("\n  Encoded value distribution:")
print(df_ispu_encoded['kategori_encoded'].value_counts().sort_index())

✓ Target variable encoded

  Encoding mapping:
    BAIK: 0
    SEDANG: 1
    TIDAK SEHAT: 2
    SANGAT TIDAK SEHAT: 3
    BERBAHAYA: 4

  Encoded value distribution:
kategori_encoded
-1.0      39
 0.0     618
 1.0    3864
 2.0     651
 3.0       4
Name: count, dtype: int64


In [19]:
# =============================================================================
# CELL 16: Handle Outliers using IQR Method for Weather Data
# =============================================================================
"""
IQR (INTERQUARTILE RANGE) METHOD:
---------------------------------
The IQR method identifies outliers as values that fall outside:
    Lower bound = Q1 - 1.5 × IQR
    Upper bound = Q3 + 1.5 × IQR

Where:
    Q1 = 25th percentile (first quartile)
    Q3 = 75th percentile (third quartile)
    IQR = Q3 - Q1

Values outside these bounds are clipped (capped) to the bounds rather than removed,
preserving the temporal continuity of the time series.
"""

def handle_outliers_iqr(df, columns, multiplier=1.5):
    """
    Handle outliers using IQR method by clipping values.
    
    Parameters:
        df: DataFrame with columns to process
        columns: List of columns to check for outliers
        multiplier: IQR multiplier for bounds (default 1.5)
    
    Returns:
        DataFrame with outliers clipped
    """
    df = df.copy()
    outlier_summary = {}
    
    for col in columns:
        if col in df.columns:
            # Calculate Q1, Q3, and IQR
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define bounds
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            
            # Count outliers before clipping
            outliers_low = (df[col] < lower_bound).sum()
            outliers_high = (df[col] > upper_bound).sum()
            
            # Clip values
            df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
            
            outlier_summary[col] = {
                'Q1': Q1, 'Q3': Q3, 'IQR': IQR,
                'lower_bound': lower_bound, 'upper_bound': upper_bound,
                'outliers_low': outliers_low, 'outliers_high': outliers_high
            }
    
    return df, outlier_summary

# Define weather columns to check for outliers
weather_numeric_cols = ['temp_max', 'temp_min', 'temp_mean', 'precipitation_sum', 
                        'wind_speed_max', 'wind_speed_mean', 'humidity_mean',
                        'pressure_mean', 'cloud_cover_mean', 'radiation_sum']

# Apply IQR outlier handling
df_weather_clean, outlier_info = handle_outliers_iqr(df_weather_with_time, weather_numeric_cols)

print("✓ Outliers handled using IQR method")
print("\n  Outlier summary:")
for col, info in outlier_info.items():
    total_outliers = info['outliers_low'] + info['outliers_high']
    if total_outliers > 0:
        print(f"    {col}: {total_outliers} outliers (low: {info['outliers_low']}, high: {info['outliers_high']})")
        print(f"      Bounds: [{info['lower_bound']:.2f}, {info['upper_bound']:.2f}]")

✓ Outliers handled using IQR method

  Outlier summary:
    temp_max: 248 outliers (low: 90, high: 158)
      Bounds: [27.25, 35.65]
    temp_min: 58 outliers (low: 46, high: 12)
      Bounds: [21.20, 26.00]
    temp_mean: 41 outliers (low: 15, high: 26)
      Bounds: [24.10, 29.70]
    precipitation_sum: 245 outliers (low: 0, high: 245)
      Bounds: [-13.45, 25.35]
    wind_speed_max: 85 outliers (low: 0, high: 85)
      Bounds: [3.10, 22.30]
    wind_speed_mean: 384 outliers (low: 0, high: 384)
      Bounds: [1.05, 11.05]
    humidity_mean: 261 outliers (low: 261, high: 0)
      Bounds: [66.00, 98.00]
    pressure_mean: 7 outliers (low: 7, high: 0)
      Bounds: [998.60, 1015.40]
    cloud_cover_mean: 175 outliers (low: 175, high: 0)
      Bounds: [25.50, 141.50]
    radiation_sum: 231 outliers (low: 231, high: 0)
      Bounds: [10.86, 27.38]


## Section 6: Final Output - Merge All Features & Scale

In [20]:
# =============================================================================
# CELL 17: Merge All Features into Master DataFrame
# =============================================================================

def create_master_dataframe(df_ispu, df_weather, df_pop_by_station, df_ndvi, df_river_agg):
    """
    Merge all processed dataframes into a single master dataframe.
    
    Parameters:
        df_ispu: Processed ISPU data with lag features
        df_weather: Processed weather data with rolling features
        df_pop_by_station: Population data by station
        df_ndvi: NDVI vegetation index data
        df_river_agg: Aggregated river quality data
    
    Returns:
        Master DataFrame with all features
    """
    # Start with ISPU data as base
    master_df = df_ispu.copy()
    
    # Merge weather data
    weather_cols_to_merge = [col for col in df_weather.columns 
                            if col not in ['year'] or col in ['tanggal', 'stasiun_id']]
    master_df = master_df.merge(
        df_weather[weather_cols_to_merge],
        on=['tanggal', 'stasiun_id'],
        how='left'
    )
    
    # Add year column for population merge
    master_df['year'] = master_df['tanggal'].dt.year
    
    # Merge population data
    df_pop_by_station = df_pop_by_station.rename(columns={'tahun': 'year'})
    master_df = master_df.merge(
        df_pop_by_station[['year', 'stasiun_id', 'jumlah_penduduk']],
        on=['year', 'stasiun_id'],
        how='left'
    )
    
    # Merge NDVI data (forward fill for dates between measurements)
    df_ndvi_clean = df_ndvi[['tanggal', 'stasiun_id', 'ndvi']].copy()
    df_ndvi_clean = df_ndvi_clean.rename(columns={'stasiun_id': 'ndvi_station'})
    # Extract station ID from NDVI data
    df_ndvi_clean['stasiun_id'] = df_ndvi_clean['ndvi_station']
    df_ndvi_clean = df_ndvi_clean.drop(columns=['ndvi_station'])
    
    # For NDVI, we do an asof merge to get the most recent NDVI value
    master_df = master_df.sort_values(['stasiun_id', 'tanggal'])
    df_ndvi_clean = df_ndvi_clean.sort_values(['stasiun_id', 'tanggal'])
    
    # Merge NDVI using merge_asof for each station
    master_dfs = []
    for station in master_df['stasiun_id'].unique():
        station_df = master_df[master_df['stasiun_id'] == station].copy()
        station_ndvi = df_ndvi_clean[df_ndvi_clean['stasiun_id'] == station].copy()
        
        if len(station_ndvi) > 0:
            merged = pd.merge_asof(
                station_df.sort_values('tanggal'),
                station_ndvi[['tanggal', 'ndvi']].sort_values('tanggal'),
                on='tanggal',
                direction='backward'
            )
            master_dfs.append(merged)
        else:
            station_df['ndvi'] = np.nan
            master_dfs.append(station_df)
    
    master_df = pd.concat(master_dfs, ignore_index=True)
    
    # Merge river quality data (use yearly average)
    df_river_yearly = df_river_agg.copy()
    df_river_yearly = df_river_yearly.rename(columns={'periode_data': 'year'})
    river_cols = ['year', 'stasiun_id', 'pH', 'BOD', 'COD', 'DO', 'TSS']
    existing_river_cols = [c for c in river_cols if c in df_river_yearly.columns]
    
    master_df = master_df.merge(
        df_river_yearly[existing_river_cols],
        on=['year', 'stasiun_id'],
        how='left'
    )
    
    return master_df

# Create master dataframe
df_master = create_master_dataframe(
    df_ispu_encoded,
    df_weather_clean,
    df_pop_by_station,
    df_ndvi,
    df_river_agg
)

print(f"✓ Master dataframe created")
print(f"  Shape: {df_master.shape}")
print(f"  Date range: {df_master['tanggal'].min().date()} to {df_master['tanggal'].max().date()}")
print(f"  Stations: {sorted(df_master['stasiun_id'].unique())}")
print(f"\n  Columns ({len(df_master.columns)}):")
print(f"  {list(df_master.columns)}")

✓ Master dataframe created
  Shape: (5175, 55)
  Date range: 2022-01-01 to 2025-08-31
  Stations: ['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']

  Columns (55):
  ['tanggal', 'stasiun_id', 'stasiun', 'pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori', 'year', 'pm_sepuluh_lag_1d', 'pm_sepuluh_lag_7d', 'pm_duakomalima_lag_1d', 'pm_duakomalima_lag_7d', 'kategori_encoded', 'temp_max', 'temp_min', 'precipitation_sum', 'precipitation_hours', 'wind_speed_max', 'wind_direction_10m_dominant', 'radiation_sum', 'temp_mean', 'humidity_mean', 'cloud_cover_mean', 'pressure_mean', 'wind_gusts_max', 'wind_direction_alt', 'humidity_max', 'humidity_min', 'cloud_cover_max', 'cloud_cover_min', 'wind_gusts_mean', 'wind_speed_mean', 'wind_gusts_min', 'wind_speed_min', 'pressure_max', 'pressure_min', 'wind_sin', 'wind_cos', 'precipitation_sum_rolling_3d_mean', 'temp_mean_rolling_3d_mean', 'month', 'is_weekend', 'is_hol

In [21]:
# =============================================================================
# CELL 18: Set Index and Apply StandardScaler
# =============================================================================

def scale_numeric_features(df, exclude_cols=None):
    """
    Apply StandardScaler to all numeric features.
    
    StandardScaler transforms features to have:
    - Mean = 0
    - Standard deviation = 1
    
    Formula: z = (x - μ) / σ
    
    Parameters:
        df: DataFrame with features
        exclude_cols: Columns to exclude from scaling
    
    Returns:
        Tuple of (scaled DataFrame, scaler object, list of scaled columns)
    """
    df = df.copy()
    
    if exclude_cols is None:
        exclude_cols = []
    
    # Add default columns to exclude (identifiers, dates, categorical)
    default_exclude = ['tanggal', 'stasiun_id', 'stasiun', 'kategori', 'kategori_encoded',
                       'parameter_pencemar_kritis', 'year', 'is_weekend', 'is_holiday_nasional', 'month']
    exclude_cols = list(set(exclude_cols + default_exclude))
    
    # Get numeric columns to scale
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cols_to_scale = [col for col in numeric_cols if col not in exclude_cols]
    
    # Apply StandardScaler
    scaler = StandardScaler()
    
    # Handle missing values before scaling
    df_to_scale = df[cols_to_scale].fillna(df[cols_to_scale].median())
    
    # Fit and transform
    scaled_values = scaler.fit_transform(df_to_scale)
    
    # Create scaled dataframe
    df_scaled = df.copy()
    for i, col in enumerate(cols_to_scale):
        df_scaled[col] = scaled_values[:, i]
    
    return df_scaled, scaler, cols_to_scale

# Set multi-index on tanggal and stasiun
df_master_indexed = df_master.set_index(['tanggal', 'stasiun_id']).sort_index()

# Apply StandardScaler
df_master_scaled, scaler, scaled_columns = scale_numeric_features(df_master)

# Set index for scaled version too
df_master_scaled_indexed = df_master_scaled.set_index(['tanggal', 'stasiun_id']).sort_index()

print("✓ Master dataframe indexed by (tanggal, stasiun)")
print(f"✓ StandardScaler applied to {len(scaled_columns)} numeric features")
print(f"\n  Scaled columns:")
print(f"  {scaled_columns}")

print(f"\n  Final master dataframe shape: {df_master_scaled_indexed.shape}")
print(f"\n  Sample of scaled data (first 5 rows):")
df_master_scaled_indexed.head()

✓ Master dataframe indexed by (tanggal, stasiun)
✓ StandardScaler applied to 41 numeric features

  Scaled columns:
  ['pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'temp_max', 'temp_min', 'precipitation_sum', 'precipitation_hours', 'wind_speed_max', 'wind_direction_10m_dominant', 'radiation_sum', 'temp_mean', 'humidity_mean', 'cloud_cover_mean', 'pressure_mean', 'wind_gusts_max', 'wind_direction_alt', 'humidity_max', 'humidity_min', 'cloud_cover_max', 'cloud_cover_min', 'wind_gusts_mean', 'wind_speed_mean', 'wind_gusts_min', 'wind_speed_min', 'pressure_max', 'pressure_min', 'wind_sin', 'wind_cos', 'precipitation_sum_rolling_3d_mean', 'temp_mean_rolling_3d_mean', 'jumlah_penduduk', 'ndvi', 'pH', 'BOD', 'COD', 'DO', 'TSS']

  Final master dataframe shape: (5175, 53)

  Sample of scaled data (first 5 rows):


,,stasiun,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,max,parameter_pencemar_kritis,kategori,...,month,is_weekend,is_holiday_nasional,jumlah_penduduk,ndvi,pH,BOD,COD,DO,TSS
tanggal,stasiun_id,,,,,,,,,,,,,,,,,,,,,
2022-01-01,DKI4,DKI4,0.315991,0.423425,0.707544,-0.219157,0.385961,-0.370514,0.432590,"PM2,5",SEDANG,...,1.0,1.0,1.0,0.616397,1.548161,0.029933,-0.138312,-0.255926,-0.317809,0.325154
2022-01-02,DKI4,DKI4,-0.681598,-0.204145,0.510553,-0.652943,0.508530,-0.843441,-0.202411,"PM2,5",SEDANG,...,1.0,1.0,0.0,0.616397,1.548161,0.029933,-0.138312,-0.255926,-0.317809,0.325154
2022-01-03,DKI4,DKI4,-0.153463,0.214235,0.773208,-0.219157,0.569814,-0.167832,0.220923,"PM2,5",SEDANG,...,1.0,0.0,0.0,0.616397,1.548161,0.029933,-0.138312,-0.255926,-0.317809,0.325154
2022-01-04,DKI4,DKI4,0.433354,1.134671,0.904535,-0.074561,1.427794,0.169973,1.152258,"PM2,5",TIDAK SEHAT,...,1.0,0.0,0.0,0.616397,1.548161,0.029933,-0.138312,-0.255926,-0.317809,0.325154
2022-01-05,DKI4,DKI4,2.545895,3.770465,1.035863,0.503821,1.672931,0.575338,3.819262,"PM2,5",TIDAK SEHAT,...,1.0,0.0,0.0,0.616397,1.548161,0.029933,-0.138312,-0.255926,-0.317809,0.325154


In [22]:
# =============================================================================
# CELL 19: Summary Statistics and Data Quality Report
# =============================================================================

def generate_data_quality_report(df):
    """
    Generate a comprehensive data quality report.
    """
    print("=" * 70)
    print("DATA QUALITY REPORT - MASTER DATAFRAME")
    print("=" * 70)
    
    print(f"\n📊 BASIC INFO:")
    print(f"   • Total records: {len(df):,}")
    print(f"   • Total features: {len(df.columns)}")
    print(f"   • Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print(f"\n📅 TEMPORAL COVERAGE:")
    if 'tanggal' in df.index.names:
        dates = df.index.get_level_values('tanggal')
    else:
        dates = df['tanggal']
    print(f"   • Date range: {dates.min().date()} to {dates.max().date()}")
    print(f"   • Total days: {(dates.max() - dates.min()).days + 1}")
    
    print(f"\n🏢 STATION COVERAGE:")
    if 'stasiun_id' in df.index.names:
        stations = df.index.get_level_values('stasiun_id').unique()
    else:
        stations = df['stasiun_id'].unique()
    for station in sorted(stations):
        if 'stasiun_id' in df.index.names:
            count = len(df.xs(station, level='stasiun_id'))
        else:
            count = len(df[df['stasiun_id'] == station])
        print(f"   • {station}: {count:,} records")
    
    print(f"\n⚠️ MISSING VALUES:")
    missing_cols = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        missing = df[col].isna().sum()
        if missing > 0:
            missing_cols.append((col, missing, missing/len(df)*100))
    
    if missing_cols:
        for col, count, pct in sorted(missing_cols, key=lambda x: -x[1])[:10]:
            print(f"   • {col}: {count:,} ({pct:.1f}%)")
    else:
        print("   • No missing values in numeric columns!")
    
    print(f"\n🎯 TARGET VARIABLE DISTRIBUTION:")
    if 'kategori_encoded' in df.columns:
        target_dist = df['kategori_encoded'].value_counts().sort_index()
        category_names = {0: 'BAIK', 1: 'SEDANG', 2: 'TIDAK SEHAT', 3: 'SANGAT TIDAK SEHAT', 4: 'BERBAHAYA', -1: 'UNKNOWN'}
        for val, count in target_dist.items():
            name = category_names.get(int(val), 'UNKNOWN')
            print(f"   • {name} ({int(val)}): {count:,} ({count/len(df)*100:.1f}%)")
    
    print("\n" + "=" * 70)

# Generate report
generate_data_quality_report(df_master_scaled_indexed)

DATA QUALITY REPORT - MASTER DATAFRAME

📊 BASIC INFO:
   • Total records: 5,175
   • Total features: 53
   • Memory usage: 3.63 MB

📅 TEMPORAL COVERAGE:
   • Date range: 2022-01-01 to 2025-08-31
   • Total days: 1339

🏢 STATION COVERAGE:
   • DKI1: 977 records
   • DKI2: 1,005 records
   • DKI3: 996 records
   • DKI4: 1,207 records
   • DKI5: 990 records

⚠️ MISSING VALUES:
   • No missing values in numeric columns!

🎯 TARGET VARIABLE DISTRIBUTION:
   • UNKNOWN (-1): 39 (0.8%)
   • BAIK (0): 618 (11.9%)
   • SEDANG (1): 3,863 (74.6%)
   • TIDAK SEHAT (2): 651 (12.6%)
   • SANGAT TIDAK SEHAT (3): 4 (0.1%)



In [23]:
# =============================================================================
# CELL 20: Export Preprocessed Data
# =============================================================================

# Export both scaled and unscaled versions
output_dir = "../processed_data/"
os.makedirs(output_dir, exist_ok=True)

# Save master dataframe (scaled)
df_master_scaled.to_csv(os.path.join(output_dir, "master_df_scaled.csv"), index=False)
print(f"✓ Saved: {output_dir}master_df_scaled.csv")

# Save master dataframe (unscaled) - useful for EDA
df_master.to_csv(os.path.join(output_dir, "master_df_unscaled.csv"), index=False)
print(f"✓ Saved: {output_dir}master_df_unscaled.csv")

# Save scaler for later use in prediction
import pickle
with open(os.path.join(output_dir, "scaler.pkl"), 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ Saved: {output_dir}scaler.pkl")

# Save target encoder
with open(os.path.join(output_dir, "target_encoder.pkl"), 'wb') as f:
    pickle.dump(target_encoder, f)
print(f"✓ Saved: {output_dir}target_encoder.pkl")

# Save column lists for reference
preprocessing_metadata = {
    'scaled_columns': scaled_columns,
    'pollutant_columns': pollutant_columns,
    'lag_columns': lag_columns,
    'lag_periods': lag_periods,
    'rolling_columns': rolling_columns,
    'window_size': window_size,
    'target_categories': ['BAIK', 'SEDANG', 'TIDAK SEHAT', 'SANGAT TIDAK SEHAT', 'BERBAHAYA'],
    'station_coords': STATION_COORDS,
    'kota_to_station_map': KOTA_TO_STATION
}

with open(os.path.join(output_dir, "preprocessing_metadata.pkl"), 'wb') as f:
    pickle.dump(preprocessing_metadata, f)
print(f"✓ Saved: {output_dir}preprocessing_metadata.pkl")

print(f"\n✅ All preprocessing complete! Files saved to: {output_dir}")

✓ Saved: ../processed_data/master_df_scaled.csv
✓ Saved: ../processed_data/master_df_unscaled.csv
✓ Saved: ../processed_data/scaler.pkl
✓ Saved: ../processed_data/target_encoder.pkl
✓ Saved: ../processed_data/preprocessing_metadata.pkl

✅ All preprocessing complete! Files saved to: ../processed_data/


In [24]:
# =============================================================================
# CELL 21: Feature Summary Table
# =============================================================================

# Create a summary of all features in the master dataframe
feature_summary = pd.DataFrame({
    'Feature': df_master_scaled.columns,
    'Type': [str(df_master_scaled[col].dtype) for col in df_master_scaled.columns],
    'Non-Null': [df_master_scaled[col].notna().sum() for col in df_master_scaled.columns],
    'Null %': [df_master_scaled[col].isna().sum() / len(df_master_scaled) * 100 for col in df_master_scaled.columns],
})

# Add category for each feature
def categorize_feature(col):
    if col in ['tanggal', 'stasiun_id', 'stasiun']:
        return 'Identifier'
    elif col in ['kategori', 'kategori_encoded']:
        return 'Target'
    elif col in pollutant_columns:
        return 'Pollutant'
    elif 'lag' in col:
        return 'Lag Feature'
    elif 'rolling' in col:
        return 'Rolling Feature'
    elif col in ['wind_sin', 'wind_cos']:
        return 'Circular Encoding'
    elif col in ['month', 'is_weekend', 'is_holiday_nasional', 'year']:
        return 'Time Feature'
    elif col in ['temp_max', 'temp_min', 'temp_mean', 'precipitation_sum', 'precipitation_hours',
                 'wind_speed_max', 'wind_speed_mean', 'humidity_mean', 'pressure_mean', 
                 'cloud_cover_mean', 'radiation_sum', 'wind_gusts_max', 'wind_direction_10m_dominant']:
        return 'Weather'
    elif col in ['pH', 'BOD', 'COD', 'DO', 'TSS']:
        return 'River Quality'
    elif col == 'ndvi':
        return 'Vegetation Index'
    elif col == 'jumlah_penduduk':
        return 'Population'
    else:
        return 'Other'

feature_summary['Category'] = feature_summary['Feature'].apply(categorize_feature)

print("📋 FEATURE SUMMARY BY CATEGORY")
print("=" * 70)
for category in feature_summary['Category'].unique():
    features_in_cat = feature_summary[feature_summary['Category'] == category]['Feature'].tolist()
    print(f"\n{category.upper()} ({len(features_in_cat)} features):")
    print(f"  {features_in_cat}")

📋 FEATURE SUMMARY BY CATEGORY

IDENTIFIER (3 features):
  ['tanggal', 'stasiun_id', 'stasiun']

POLLUTANT (7 features):
  ['pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max']

OTHER (11 features):
  ['parameter_pencemar_kritis', 'wind_direction_alt', 'humidity_max', 'humidity_min', 'cloud_cover_max', 'cloud_cover_min', 'wind_gusts_mean', 'wind_gusts_min', 'wind_speed_min', 'pressure_max', 'pressure_min']

TARGET (2 features):
  ['kategori', 'kategori_encoded']

TIME FEATURE (4 features):
  ['year', 'month', 'is_weekend', 'is_holiday_nasional']

LAG FEATURE (4 features):
  ['pm_sepuluh_lag_1d', 'pm_sepuluh_lag_7d', 'pm_duakomalima_lag_1d', 'pm_duakomalima_lag_7d']

WEATHER (13 features):
  ['temp_max', 'temp_min', 'precipitation_sum', 'precipitation_hours', 'wind_speed_max', 'wind_direction_10m_dominant', 'radiation_sum', 'temp_mean', 'humidity_mean', 'cloud_cover_mean', 'pressure_mean', 'wind_gusts_max', 'wind_speed_mean']

CIRCULA